In [1]:
import pandas as pd
import numpy as np
#import tensorflow as tf
from tqdm import tqdm

In [2]:
from boto.s3.connection import S3Connection
from boto.s3.key import Key
import sys, os
import re
import json
from urllib.request import FancyURLopener
import cv2
import glob
import zipfile

In [3]:
KEY_FILE = '..//..//..//data//aws_keys//keys.txt'

In [4]:
f = open(KEY_FILE,'rb')
key_data = str(f.read())
keys = re.findall(pattern='\=([a-zA-Z0-9]*)',string=key_data)

In [5]:
AWS_KEY = keys[0]
AWS_SECRET = keys[1]

In [6]:
del keys

In [7]:
BUCKET = 'imaterialist-challenge'
PATH_TO_DATA = '..//..//..//data//imaterialist//'

In [8]:
conn = S3Connection(AWS_KEY, AWS_SECRET)
bucket = conn.get_bucket(BUCKET)

In [9]:
bucket_list = bucket.list()

In [21]:
files = []

True

In [22]:
for l in tqdm(bucket_list):
    keyString = str(l.key)
    # check if file exists locally, if not: download it
    if not os.path.exists(PATH_TO_DATA+keyString) and not 'imageTrain' in keyString:
        l.get_contents_to_filename(PATH_TO_DATA+keyString)
    if not 'imageTrain' in keyString:
        files.append(keyString)

35708it [00:34, 1037.13it/s]


In [23]:
json_data = []

In [24]:
set(files)

{'fgvc4_iMat.label_map.json',
 'fgvc4_iMat.task_map.json',
 'fgvc4_iMat.test.samplesubmission.csv',
 'fgvc4_iMat.train.data.json',
 'fgvc4_iMat.validation.data.json'}

In [25]:
def de_dupe_list(seq): # Order preserving
    seen = set()
    return [x for x in seq if x not in seen and not seen.add(x)]

In [26]:
for file in tqdm(de_dupe_list(files)):
    try:
        json_data.append(json.loads(open(PATH_TO_DATA+file).read()))
    except:
        print('not csv file')

  0%|          | 0/5 [00:00<?, ?it/s]

not csv file


100%|██████████| 5/5 [00:00<00:00, 12.53it/s]


# get keys

In [27]:
[k for k in json_data[2]]

['annotations', 'images', 'info', 'license']

In [28]:
train_data = json_data[2]

In [29]:
train_data['images'][0]

{'imageId': '1',
 'url': ['http://im28.gulfup.com/2012-04-14/1334424802942.jpg',
  'http://productshots0.modcloth.com/productshots/0062/9433/9c957ccc31e39e90c774dfd8351df5e7.jpg?1298662748',
  'http://productshots2.modcloth.com/productshots/0062/9433/9c957ccc31e39e90c774dfd8351df5e7.jpg?1298662748',
  'http://productshots3.modcloth.com/productshots/0062/9433/9c957ccc31e39e90c774dfd8351df5e7.jpg',
  'http://productshots3.modcloth.net/productshots/0062/9433/9c957ccc31e39e90c774dfd8351df5e7.jpg',
  'http://productshots3.modcloth.net/productshots/0062/9433/9c957ccc31e39e90c774dfd8351df5e7.jpg?1298662748',
  'http://www.canaryrook.com/uploads/products/poema-moments-to-remember-dress_mc509151449.jpg%3F1298662748',
  'https://s-media-cache-ak0.pinimg.com/originals/69/ac/48/69ac48e09b6b42ceceec674504306f63.jpg']}

In [30]:
#url = 'http://productshots0.modcloth.com/productshots/0062/9433/9c957ccc31e39e90c774dfd8351df5e7.jpg?1298662748'
url = 'http://im28.gulfup.com/2012-04-14/1334424802942.jpg'

In [31]:
imageId = '1'

In [32]:
class MyOpener(FancyURLopener):
    version = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11'
 
myopener = MyOpener()

/home/ubuntu/anaconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:4: DeprecationWarning: MyOpener style of invoking requests is deprecated. Use newer urlopen functions/methods
  after removing the cwd from sys.path.


In [33]:
cv_img = cv2.imread(PATH_TO_DATA+imageId+'.jpg')

In [34]:
np.shape(cv_img)

(500, 350, 3)

In [35]:
#images.append([ImageLocationToInt(img),cv2.resize(cv_img, (new_dimensions[0],new_dimensions[1]))])

In [36]:
image_data_clean = []
image_data_anomalies = []

# handling timeouts

In [37]:
from functools import wraps
import errno
import os
import signal


In [38]:
class TimeoutError(Exception):
    pass

def timeout(seconds=10, error_message=os.strerror(errno.ETIME)):
    def decorator(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout)
            signal.alarm(seconds)
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return wraps(func)(wrapper)

    return decorator

In [39]:
timeout_param = 1

In [40]:
@timeout(timeout_param)
def getImageFromUrl(url,store_path):
    return(myopener.retrieve(url, PATH_TO_DATA+imageId+'.jpg'))


In [41]:
import time

In [42]:
@timeout(1)
def tryFunction():
    time.sleep(1.1)
    return(1)


In [43]:
retrieved_files = glob.glob(PATH_TO_DATA+"*.jpg")

In [44]:
img_to_string = lambda x: re.sub(pattern='.jpg',repl='',string=re.findall(pattern='[0-9]*.jpg',string=x)[0])

In [45]:
first_img = np.max([float(img_to_string(img_file)) for img_file in retrieved_files])

In [35]:
for image in tqdm(train_data['images']):
    imageId = image['imageId']
    if not os.path.exists(PATH_TO_DATA+imageId+".jpg") and float(imageId) >= first_img:
        for url in image['url']:
            try:
                # download image
                getImageFromUrl(url, PATH_TO_DATA+imageId+'.jpg')
                # retrieve image
                cv_img = cv2.imread(PATH_TO_DATA+imageId+'.jpg')
                # append to data structure
                image_data_clean.append([imageId,cv_img])
                break
            except:
                image_data_anomalies.append(imageId)

100%|██████████| 42029/42029 [1:14:31<00:00,  1.92it/s]  


In [46]:
stored_image_ids = [float(img_to_string(img_file)) for img_file in retrieved_files]

In [47]:
stored_image_ids_set = set(stored_image_ids)

In [48]:
max_id = np.max(stored_image_ids)

In [59]:
max_id

42029.0

In [49]:
missing_ids = [ x for x in list(range(int(max_id))) if x not in stored_image_ids_set ]

In [50]:
len(missing_ids)

1078

# 1077 missing ids (0 doesn't count)

In [51]:
missing_ids[:10]

[0, 56, 148, 213, 258, 272, 314, 315, 352, 403]

# Checking ID: 56, image doesn't exist anymore

# Upload data to s3

In [52]:
retrieved_files = glob.glob(PATH_TO_DATA+"*.jpg")

In [53]:
sample_file = retrieved_files[0]

In [54]:
to_s3_file = lambda x: 'imageTrain//' + re.findall(pattern='[0-9]*.jpg',string=x)[0]

In [60]:
s3_file = to_s3_file(sample_file)

# It's free to transfer data into s3

In [61]:
# https://aws.amazon.com/s3/pricing/?sc_channel=PS&sc_campaign=acquisition_AU&sc_publisher=google&sc_medium=s3_b&sc_content=sitelink&sc_detail=amazons3&sc_category=s3&sc_segment=pricing&sc_matchtype=p&sc_country=AU&s_kwcid=AL!4422!3!80141814084!p!!g!!amazons3&ef_id=WFUQZgAAAHA@kieg:20170606095940:s

In [66]:
keystrings = [str(l.key) for l in tqdm(bucket_list)]

35708it [00:31, 1148.76it/s]


In [68]:
keystrings = set(keystrings)

In [70]:
s3_file.replace('//','/') in keystrings

True

In [71]:
# yolo: transfer remaining files in.
for sample_file in tqdm(retrieved_files):
    s3_file = to_s3_file(sample_file)
    if not s3_file.replace('//','/') in keystrings:
        k = Key(bucket)
        k.key = s3_file
        k.set_contents_from_filename(filename=sample_file)

100%|██████████| 40952/40952 [47:33<00:00,  1.44it/s]    


# To zip the folder:

In [52]:
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

854074